In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tnrange, tqdm_notebook
import gc
import operator

In [2]:
import warnings
warnings.filterwarnings('ignore', message='Changing the shape of non-C contiguous array')

In [3]:
pd.set_option('display.max_columns', 500)

In [4]:
sns.set_context('talk')

In [5]:
import average_precision

In [6]:
import datetime

# Thoughts

Нам нужно предсказать для каждого пользователя пять категорий в порядке убывания, на товарные предложения внутри которых он кликал чаще всего за восьмую неделю. 

Можно выделить седьмую неделю и валидироваться по аггрегированным кликам для неё.

Для начала будем работать только с train_clicks и train_category_views. Только с полями user_id, category_id, day

In [7]:
views = pd.read_csv('data/train_category_views.csv', parse_dates=['day'])
clicks = pd.read_csv('data/train_clicks.csv', parse_dates=['day'])[views.columns]

In [8]:
test_users = pd.read_csv('data/test_users.csv')

In [9]:
clicks.head()

,user_id,category_id,day
0,46,672,2016-08-04
1,48,170,2016-08-04
2,48,170,2016-08-04
3,53,1190,2016-08-04
4,93,56,2016-08-04


In [10]:
clicks.day.max() - clicks.day.min()

Timedelta('52 days 00:00:00')

In [11]:
clicks.category_id.max()

2653

In [12]:
views.category_id.max()

2682

Переведём данные в "клики (просмотры) в день пользователя по категории"

In [13]:
agg_views = views.copy()
agg_views['count'] = 1
agg_views = agg_views.groupby(['user_id', 'day', 'category_id']).count().sort_index()

In [14]:
agg_clicks = clicks.copy()
agg_clicks['count'] = 1
agg_clicks = agg_clicks.groupby(['user_id', 'day', 'category_id']).count().sort_index()

In [15]:
(clicks.groupby('user_id').day.max() - clicks.groupby('user_id').day.min()).median()

Timedelta('0 days 00:00:00')

In [16]:
(views.groupby('user_id').day.max() - views.groupby('user_id').day.min()).median()

Timedelta('0 days 00:00:00')

In [17]:
clicks.groupby('user_id')['category_id'].count().median()

2.0

In [18]:
clicks.groupby('user_id')['category_id'].count().mean()

3.9594260406245434

In [19]:
(clicks.groupby('user_id')['category_id'].count() > 4).sum()

38643

In [20]:
clicks['user_id'].nunique()

177922

А давайте мягко предсказывать категорию, на которую больше всего накликает пользователь. Как ответ будет давать упорядоченную пятёрку топ-5 предсказаний. Как y_train будет soft_max от накликанного

Можно пытаться предсказать поведение среднего пользователя.

In [21]:
len(set(test_users.user_id) - set(clicks.user_id))

0

Все пользователи из теста есть в трейне

In [22]:
test_users.head()

,user_id
0,8
1,12
2,27
3,39
4,40


In [23]:
clicks

,user_id,category_id,day
0,46,672,2016-08-04
1,48,170,2016-08-04
2,48,170,2016-08-04
3,53,1190,2016-08-04
4,93,56,2016-08-04
5,106,96,2016-08-04
6,106,96,2016-08-04
7,113,2124,2016-08-04
8,113,2124,2016-08-04
9,113,2124,2016-08-04


Выделим седьмую неделю

In [ ]:
def split_by_date(df, date):
    

In [ ]:
train_agg_views, validation_agg_views = split_by_date(agg_views, views.day.max() - datetime.timedelta(6))

In [14]:
user_profile = pd.read_csv('data/train_user_profile.csv', parse_dates=['day'])

In [15]:
user_profile.user_id.nunique()

53428

In [12]:
test_users.shape

(31712, 1)

In [16]:
len(set(test_users.user_id) - set(user_profile.user_id))

22120

Чёт всё гавно какое-то. Возьмём бейзлайн, и потюним его

In [25]:
clicks.day.dt.dayofyear

0         217
1         217
2         217
3         217
4         217
5         217
6         217
7         217
8         217
9         217
10        217
11        217
12        217
13        217
14        217
15        217
16        217
17        217
18        217
19        217
20        217
21        217
22        217
23        217
24        217
25        217
26        217
27        217
28        217
29        217
         ... 
704439    269
704440    269
704441    269
704442    269
704443    269
704444    269
704445    269
704446    269
704447    269
704448    269
704449    269
704450    269
704451    269
704452    269
704453    269
704454    269
704455    269
704456    269
704457    269
704458    269
704459    269
704460    269
704461    269
704462    269
704463    269
704464    269
704465    269
704466    269
704467    269
704468    269
Name: day, dtype: int64

In [12]:
number_of_categories = max(max(clicks.category_id), max(views.category_id))
number_of_users = max(max(clicks.user_id), max(views.user_id))

In [13]:
user_clicks = np.zeros((number_of_users + 1, number_of_categories + 1))
for row in clicks.iterrows():
    _, row = row
    user_clicks[row['user_id'], row['category_id']] += np.exp((row.day.dayofyear - 269) / 14)

In [14]:
user_clicks.sum()

192507.89925929814

In [15]:
user_clicks[test_users.values.reshape(-1), :].sum()

101437.18588566085

In [16]:
user_clicks.shape

(200000, 2683)

divider = user_clicks.sum(axis=1).reshape((-1, 1))
divider = np.where(divider > 0, divider, 1)

user_clicks /= divider

In [17]:
average_clicks = user_clicks.sum(axis=0)

In [18]:
average_clicks /= user_clicks.shape[0]

In [19]:
user_clicks += average_clicks / 10

In [24]:
number_of_categories = max(max(clicks.category_id), max(views.category_id))
number_of_users = max(max(clicks.user_id), max(views.user_id))

In [21]:
user_views = np.zeros((number_of_users + 1, number_of_categories + 1))
for row in views.iterrows():
    _, row = row
    user_views[row['user_id'], row['category_id']] += np.exp((row.day.dayofyear - 269) / 14)

In [22]:
user_clicks.sum()

211758.68918522942

In [23]:
user_views.sum()

214203.62992592141

In [24]:
user_clicks += user_views / 10

In [38]:
user_clicks = np.zeros((number_of_users + 1, number_of_categories + 1))

In [39]:
test_categories = np.argsort(-user_clicks[test_users.user_id.values.reshape(-1), :], axis=1)[:, :5]

In [40]:
test_categories[:5]

array([[   0, 1784, 1785, 1786, 1787],
       [   0, 1784, 1785, 1786, 1787],
       [   0, 1784, 1785, 1786, 1787],
       [   0, 1784, 1785, 1786, 1787],
       [   0, 1784, 1785, 1786, 1787]])

Найдём порядок на нулях

In [25]:
dummy_order = np.argsort(-np.zeros(number_of_categories + 1))

In [26]:
dummy_order

array([   0, 1784, 1785, ...,  898,  900, 2682])

Можно посмотреть, как зайдёт такая лажа на лидерборде

In [27]:
def join_categories(row):
    base_str = ' '.join(map(str, row))
    return base_str + ' '*(25 - len(base_str))

In [28]:
test_users['categories'] = np.apply_along_axis(join_categories, 1, test_categories)

In [28]:
test_users['categories'] = np.apply_along_axis(join_categories, 0, dummy_order[:5])

In [29]:
test_users.head()

,user_id,categories
0,8,0 1784 1785 1786 1787
1,12,0 1784 1785 1786 1787
2,27,0 1784 1785 1786 1787
3,39,0 1784 1785 1786 1787
4,40,0 1784 1785 1786 1787


In [30]:
test_users.to_csv('csv/baseline_dummy.csv', index=None)

Пока есть два параметра -- затухание и сглаживание средним